In [1]:
import numpy as np
from pymatgen.io.cif import CifParser
from pymatgen.core import Structure
from pymatgen.core.lattice import Lattice

In [2]:
import sys
sys.path.append('../..')
from ai4mat.data.data import (
    StorageResolver)

In [3]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

In [4]:
mp_cell = CifParser(StorageResolver()["materials"] / 
    "InSe_mp-1079260_computed.cif").get_structures(primitive=False)[0]

/home/kna/.cache/pypoetry/virtualenvs/2d-defects-potential-learning-pYjw2mkT-py3.10/lib64/python3.10/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [5]:
selection = (mp_cell.frac_coords[:, 2] < 0.7) & (mp_cell.frac_coords[:, 2] > 0.3)
assert np.count_nonzero(selection)*2 == len(mp_cell)

In [6]:
view(AseAtomsAdaptor.get_atoms(mp_cell), viewer='ngl')

In [7]:
# Create a new Structure object with the selected atoms
monolayer = Structure.from_sites([mp_cell[i] for i in np.where(selection)[0]])
lattice_np = np.array(monolayer.lattice.matrix)
lattice_np[2, 2] = 20
tall_lattice = Lattice(lattice_np)

reference_cell = Structure(
    lattice=tall_lattice,
    species=monolayer.species,
    coords=monolayer.cart_coords,
    coords_are_cartesian=True,
)

In [8]:
reference_cell

Structure Summary
Lattice
    abc : 4.09647352 4.09647352 20.0
 angles : 90.0 90.0 119.99999231
 volume : 290.65711918649475
      A : 4.09647352 0.0 2.5083665920299456e-16
      B : -2.048236283849211 3.547650409156004 2.5083665920299456e-16
      C : 0.0 0.0 20.0
    pbc : True True True
PeriodicSite: In (0.0000, 2.3651, 10.5888) [0.3333, 0.6667, 0.5294]
PeriodicSite: In (0.0000, 2.3651, 7.7547) [0.3333, 0.6667, 0.3877]
PeriodicSite: Se (2.0482, 1.1826, 11.8563) [0.6667, 0.3333, 0.5928]
PeriodicSite: Se (2.0482, 1.1826, 6.4872) [0.6667, 0.3333, 0.3244]

In [9]:
from pymatgen.io.cif import CifWriter
CifWriter(reference_cell).write_file(StorageResolver()["materials"] /"InSe.cif")

Structure appears to have two layers https://www.researchgate.net/figure/Basic-properties-of-InSe-monolayer-a-Top-and-side-view-of-InSe-monolayer-b-Band_fig1_324591452

In [10]:
supercell = reference_cell.copy()
supercell.make_supercell((6, 6, 1))

In [11]:
view(AseAtomsAdaptor.get_atoms(supercell), viewer='ngl')

In [12]:
mp_cell

Structure Summary
Lattice
    abc : 4.09647352 4.09647352 18.343448
 angles : 90.0 90.0 119.99999231
 volume : 266.5826875813634
      A : 4.09647352 0.0 2.5083665920299456e-16
      B : -2.048236283849211 3.547650409156004 2.5083665920299456e-16
      C : 0.0 0.0 18.343448
    pbc : True True True
PeriodicSite: In (0.0000, 2.3651, 10.5888) [0.3333, 0.6667, 0.5773]
PeriodicSite: In (0.0000, 2.3651, 7.7547) [0.3333, 0.6667, 0.4228]
PeriodicSite: In (2.0482, 1.1826, 16.9286) [0.6667, 0.3333, 0.9229]
PeriodicSite: In (2.0482, 1.1826, 1.4148) [0.6667, 0.3333, 0.0771]
PeriodicSite: Se (0.0000, 0.0000, 15.6599) [0.0000, 0.0000, 0.8537]
PeriodicSite: Se (0.0000, 0.0000, 2.6836) [0.0000, 0.0000, 0.1463]
PeriodicSite: Se (2.0482, 1.1826, 11.8563) [0.6667, 0.3333, 0.6463]
PeriodicSite: Se (2.0482, 1.1826, 6.4872) [0.6667, 0.3333, 0.3537]

In [13]:
len(supercell)/2* 0.062501

4.500072

In [14]:
len(supercell)/2

72.0

In [15]:
0.125/2

0.0625

In [16]:
0.125/2

0.0625

In [17]:
5/(len(supercell)/2)

0.06944444444444445